In [1]:
import math
from interval import interval, imath, fpu

In [2]:
class Taylor:
    def __init__(self, coef:list, order:int):
        self.order=order
        if len(coef)<(order+1):
            for i in range(order+1-len(coef)):
                coef.append(0)
        self.coef=coef

    def __repr__(self):
        return 'Taylor(%s, %d)'%(self.coef, self.order)
   
    def fixshape(x, y):
        order=max(x.order,y.order)
        x1=x.coef[:]
        y1=y.coef[:]
        if x.order<order:
            for i in range(order-x.order):
                x1.append(0)
        else:
            for i in range(order-y.order):
                y1.append(0)
        return Taylor(x1,order), Taylor(y1,order), order
        
    def fixshape1(x, y):
        order=x.order+y.order
        x1=x.coef[:]
        y1=y.coef[:]
        for i in range(order-x.order):
            x1.append(0)
        for j in range(order-y.order):
            y1.append(0)
        return Taylor(x1,order), Taylor(y1,order), order   
    
    def firstnonzero(self):
        order=self.order
        nonzero=order+1
        for i in range(order+1):
            if self.coef[i]!=0:
                nonzero=i
                break
        return nonzero
    
    def __add__(self, other):
        coef=[]
        if isinstance(other,(int,float)):
            other=Taylor([other],0)
        self1, other1, order=self.fixshape(other)
        for i in range(order+1):
            coef.append(self1.coef[i]+other1.coef[i])
        return Taylor(coef,order)
    
    def __radd__(self,other):
        return self+other
    
    def __sub__(self, other):
        coef=[]
        if isinstance(other,(int,float)):
            other=Taylor([other],0)
        self1, other1, order=self.fixshape(other)
        for i in range(order+1):
            coef.append(self1.coef[i]-other1.coef[i])
        return Taylor(coef,order)
    
    def __rsub__(self,other):
        if isinstance(other,(int,float)):
            other=Taylor([other],0)
        return other-self
    
    def __mul__(self, other):
        coef=[]
        if isinstance(other,(int,float)):
            if other==0:
                return 0
            else:
                other=Taylor([other],0)
        self1, other1, order=Taylor.fixshape1(self, other)
        coef.append(self1.coef[0]*other1.coef[0])
        for i in range(1,order+1):
            s=0
            for j in range(i+1):
                s+=self1.coef[j]*other1.coef[i-j]
            coef.append(s)
        return Taylor(coef,order)
    
    def __rmul__(self,other):
        return self*other
    
    def __truediv__(self, other):
        if isinstance(other,(int,float)):
            other=Taylor([other],0)
        self1, other1, order=self.fixshape(other)
        coef=Taylor.zero(order).coef
        orddivfact, cdivfact=self1.divfactorization(other1)
        coef[0]=cdivfact
        for i in range(orddivfact+1,order+1):
            coef[i-orddivfact]=self.divcoef(i,self1.coef,other1.coef,coef,orddivfact)
        return Taylor(coef,order)
    
    def divfactorization(self,other):
        a1nz=self.firstnonzero()
        b1nz=other.firstnonzero()
        orddivfact=min(a1nz,b1nz)
        try:
            cdivfact=self.coef[orddivfact]/other.coef[orddivfact]
            aux=cdivfact**2
        except ZeroDivisionError:
            print("Division does not define a Taylor polynomial\n")
        return orddivfact, cdivfact
    
    def divcoef(self,i,coef1,coef2,coef,orddivfact):
        s=0
        for j in range(i+1):
            s+=coef[j]*coef2[i-j]
        s=(coef1[i]-s)/coef2[orddivfact]
        return s

    def zero(n):
        coef=[]
        coef.append(0)
        for i in range(n):
            coef.append(0)
        return Taylor(coef,n)
    
    def one(n):
        coef=[]
        coef.append(1)
        for i in range(n):
            coef.append(0)
        return Taylor(coef,n)
    
    def __eq__(self,other):
        self1, other1, order=Taylor.fixshape(self, other)
        return self1.coef==other1.coef
    
    def square(self):
        order=self.order*2
        for a in range(self.order):
            self.coef.append(0)
        coef=[]
        coef.append((self.coef[0])**2)
        for i in range(1,order+1):
            coef.append(self.squarecoef(i,self.coef))
        return Taylor(coef,order)        
    
    def squarecoef(self,i,coef):
        s=0
        iend=int((i-2+i%2)/2)
        for j in range(iend+1):
            s+=coef[j]*coef[i-j]
        s=2*s
        if i%2==0:  
            s+=coef[int(i/2)]**2
        return s
    
    def __pow__(self,n):
        if type(n)==int:
            order=self.order*n
            for a in range(self.order):
                self.coef.append(0)
            if n<0:
                return Taylor.one(n)/self.__pow__(-n)
            elif n==0:
                return Taylor.one(order)
            elif n%2==0:
                if n==2:
                    return self.square()
                else:
                    p=int(n/2)
                    return Taylor(self.__pow__(p).square().coef,order)
            elif n==1:
                return self
            else:
                p=int((n-1)/2)
                return Taylor((self*self.__pow__(p).square()).coef,order)
        elif type(n)==float:
            order=self.order
            uno=Taylor.one(order)
            if n==0:
                return uno
            elif n==0.5:
                return self.sqrt()
            l0nz=self.firstnonzero()
            lnull=n*l0nz
            lnull=int(lnull)
            aux=(self.coef[l0nz])**n
            coef=Taylor.zero(order).coef
            coef[lnull]=aux
            k0=lnull+l0nz
            for i in range(k0+1,order+1):
                coef[i-l0nz]=self.powcoef(i,self.coef,n,coef,l0nz)
            return Taylor(coef,order)

    def powcoef(self,i,ac,x,coef,l0nz):
        s=0
        for j in range(i-l0nz):
            aux=x*(i-j)-j
            s+=aux*ac[i-j]*coef[j]
        aux=i-l0nz*(x+1)
        s=s/(aux*ac[l0nz])
        return s
    
    def diffTaylor(self):
        order=self.order
        coef=[]
        coef.append(self.coef[1])
        for i in range(2,order+1):
            coef.append(i*self.coef[i])
        coef.append(0)
        return Taylor(coef,order)
        
    def inteTaylor(self,x):
        order=self.order
        coef=[]
        coef.append(x)
        for i in range(1,order+1):
            coef.append(self.coef[i-1]/i)
        return Taylor(coef,order)
     
    def __call__(self,dx):
        order=self.order
        s=self.coef[-1]
        for i in range(order-1,-1,-1):
            s=s*dx+self.coef[i]
        return s
        
    def deriv(self,n):
        s=math.factorial(n)*self.coef[n]
        return s
    
    #f(g(x)),self=f,other=g
    def compose(self,other):
        s=self.coef[0]
        for i in range(1,self.order+1):
            s+=self.coef[i]*other**i
        return s

        
        

In [3]:
a=Taylor([0,1,0,0,0],4)
b=Taylor([1,2,3],2)
c=Taylor([0,1,2,3,4],4)

In [4]:
a+b

Taylor([1, 3, 3, 0, 0], 4)

In [5]:
(a+b)(1)

7

In [6]:
2+a

Taylor([2, 1, 0, 0, 0], 4)

In [7]:
a-b

Taylor([-1, -1, -3, 0, 0], 4)

In [8]:
a-2

Taylor([-2, 1, 0, 0, 0], 4)

In [9]:
2-a

Taylor([2, -1, 0, 0, 0], 4)

In [10]:
b*c

Taylor([0, 1, 4, 10, 16, 17, 12], 6)

In [11]:
2*b

Taylor([2, 4, 6], 2)

In [12]:
0*b

0

In [13]:
c/2

Taylor([0.0, 0.5, 1.0, 1.5, 2.0], 4)

In [14]:
a*(b+1)

Taylor([0, 2, 2, 3, 0, 0, 0], 6)

In [15]:
b.coef[0]+2*c+3*c**2

Taylor([1, 2, 7, 18, 38, 60, 75, 72, 48], 8)

In [16]:
b.compose(c)

Taylor([1, 2, 7, 18, 38, 60, 75, 72, 48], 8)

In [17]:
(lambda x: x**2 + x + x**3 - x**2)(interval[0,3])

interval([-9.0, 39.0])

In [18]:
f1=Taylor([0,1,1],2)
f2=Taylor([0,0,-1,1],3)
f3=f1+f2
f3(interval[0,3])

interval([0.0, 30.0])

In [19]:
(lambda x: x**2 + x + x**3 - x**2)(interval[0,3])

interval([-9.0, 39.0])

In [20]:
(lambda x: x**2 + x + x**3 - x**2)(interval[0,2])

interval([-4.0, 14.0])

In [21]:
(lambda x: x**2 + x + x**3 - x**2)(interval[2,3])

interval([5.0, 35.0])

In [22]:
(lambda x: x**2 + x + x**3 - x**2)(interval[0,2])|(lambda x: x**2 + x + x**3 - x**2)(interval[2,3])

interval([-4.0, 35.0])